In [1]:
import os
import numpy as np
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import tensorflow as tf
import tensorflow.keras as tfk
from sklearn.metrics import *
import copy
import numpy as np
import matplotlib.pyplot as plt
import pickle

2023-04-30 15:14:08.747098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    #strategy = tf.distribute.MirroredStrategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:

num_aminoAcids = {0:'A', 1:'C', 2:'E', 3:'D', 4:'G', 5:'F', 6:'I', 7:'H', 8:'K', 9:'M', 10:'L',
            11:'N', 12:'Q', 13:'P', 14:'S', 15:'R', 16:'T', 17:'W', 18:'V', 19:'Y', 20:'X'}
embedding_keys = {
        '<pad>':0,
        'A':1,
        'C':2,
        'D':3,
        'E':4,
        'F':5,
        'G':6,
        'H':7,
        'I':8,
        'K':9,
        'L':10,
        'M':11,
        'N':12,
        'P':13,
        'Q':14,
        'R':15,
        'S':16,
        'T':17,
        'V':18,
        'W':19,
        'X':20,
        'Y':21,
        'B':22,
        'U':23,
        'Z':24,
        'O':25,
        '<mask>':26}
embedding_list = list(embedding_keys)

In [4]:
class NonZeroCCE(tf.keras.losses.Loss):
    def __init__(self,Weights=[],smoothing = None):
        super().__init__()
        if smoothing is not None:
            self.cce = tf.keras.losses.CategoricalCrossentropy(reduction='none', label_smoothing=smoothing)
        else:
            self.cce = tf.keras.losses.CategoricalCrossentropy(reduction='none')
        self.weights = np.array(Weights)
    def call(self, y_true, y_pred):
        if len(self.weights)>0:
            loss = self.cce(self.weights*y_true,y_pred)
        else:
            loss = self.cce(y_true,y_pred)
        return tf.math.reduce_sum(loss)/((y_true.shape[0]*y_true.shape[1])-float(tf.math.reduce_sum(tf.cast(tf.equal(tf.math.reduce_sum(y_true,-1),0),tf.int32))))

def Accuracy(y_true,y_pred):
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()

    m = np.sum(y_true, axis=-1)
    y_t = np.argmax(y_true[m==1],axis=-1)
    y_p = np.argmax(y_pred[m==1],axis=-1)
    return accuracy_score(y_t,y_p)

def to_q3(x):
    y = []
    for i in x:
        if i in [0,6,7]:
            y += [1]
        elif i in [1,2]:
            y += [2]
        else:
            y += [3]
    return y

In [5]:
#Dataset for SSP
num_aminoAcids = {0:'A', 1:'C', 2:'E', 3:'D', 4:'G', 5:'F', 6:'I', 7:'H', 8:'K', 9:'M', 10:'L',
            11:'N', 12:'Q', 13:'P', 14:'S', 15:'R', 16:'T', 17:'W', 18:'V', 19:'Y', 20:'X'}
num_ss = {0:'L',1:'B',2:'E',3:'G',4:'I',5:'H',6:'S',7:'T'}
aminoAcid_I = {j:i+1 for i,j in num_aminoAcids.items()}
aminoAcid_I['<pad>'] = 0
aminoAcid_I['<S>'] = len(aminoAcid_I)
aminoAcid_I['<EOS>'] = len(aminoAcid_I)
ss_I = {j:i+1 for i,j in num_ss.items()}
ss_I['<pad>'] = 0
ss_I['X'] = len(ss_I)
ss_I['<S>'] = len(ss_I)
ss_I['<EOS>'] = len(ss_I)

trainDataPath = './Data/Secondary_Structure_Train_Dataset.npz'
testDataPath = './Data/Secondary_Structure_Test_Dataset.npz'

tmp = np.load('./Data/Secondary_Structure_Motif_Antimotif.npz')
motifs = tmp['motifs']
antiMotifs = tmp['antimotifs']
len(motifs),len(antiMotifs)
def load_data(file_path):
    data = np.load(file_path)
    sequences = data['sequences']
    pssms = data['pssms']
    secondary_structure = data['secondaryStrucs']

    in1 = np.zeros((sequences.shape[0], 702), dtype=np.int32)
    in2 = np.zeros((sequences.shape[0], 702,22), dtype=np.float32)
    in3 = np.zeros((sequences.shape[0], 702), dtype=np.int32)
    out = np.zeros((sequences.shape[0], 702), dtype=np.int32)
    for i in tqdm(range(sequences.shape[0])):
        seq = '-'
        in1[i,0] = aminoAcid_I['<S>']
        in3[i,0] = 1
        out[i,0] = ss_I['<S>']
        for j in range(sequences.shape[1]):
            if np.sum(sequences[i,j,:]) == 0:
                in1[i,j+1] = aminoAcid_I['<EOS>']
                in3[i,j+1] = j+2
                out[i,j+1] = ss_I['<EOS>']
                break
            in3[i,j+1] = j+2
            t = num_aminoAcids[np.argmax(sequences[i,j,:])]
            seq += t
            in1[i,j+1] = aminoAcid_I[t]
            out[i,j+1] = ss_I[num_ss[np.argmax(secondary_structure[i,j,:])]]
            if np.sum(secondary_structure[i,j,:]) == 0:
                out[i,j+1] = ss_I['X']
            in2[i,j+1] = pssms[i,j]
    in4 = np.where(in1!=0, 1, 0)[:,:,None]
    return in1, in2, in3, in4, out
X1, X2, X3, X4, Y = load_data(trainDataPath)
X1_val = X1[12000:]
X2_val = X2[12000:]
X3_val = X3[12000:]
X4_val = X4[12000:]
Y_val = tf.one_hot(Y[12000:], 9)[:,:,1:]

X1 = X1[:12000]
X2 = X2[:12000]
X3 = X3[:12000]
X4 = X4[:12000]
Y = tf.one_hot(Y[:12000], 9)[:,:,1:]

X1_te, X2_te, X3_te, X4_te, Y_te = load_data(testDataPath)
Y_te = tf.one_hot(Y_te, 9)[:,:,1:]

X1_temp = []
for seq in X1:
    end = np.where(seq==23)[0]
    if len(end)==0:
        end = len(seq)
    else:
        end = end[0]
    X1_temp.append([embedding_keys[num_aminoAcids[seq[idx]-1]] for idx in range(1,end)])
    X1_temp[-1].extend([0]*(2000-len(X1_temp[-1])))
    X1_temp[-1] = np.array(X1_temp[-1])
X1_temp = np.array(X1_temp)


X1_val_temp = []
for seq in X1_val:
    end = np.where(seq==23)[0]
    if len(end)==0:
        end = len(seq)
    else:
        end = end[0]
    X1_val_temp.append([embedding_keys[num_aminoAcids[seq[idx]-1]] for idx in range(1,end)])
    X1_val_temp[-1].extend([0]*(2000-len(X1_val_temp[-1])))
    X1_val_temp[-1] = np.array(X1_val_temp[-1])
X1_val_temp = np.array(X1_val_temp)

X1_te_temp = []
i=0
for seq in X1_te:
    end = np.where(seq==23)[0]
    if len(end)==0:
        end = len(seq)-1
    else:
        end = end[0]
    X1_te_temp.append([embedding_keys[num_aminoAcids[seq[idx]-1]] for idx in range(1,end)])
    X1_te_temp[-1].extend([0]*(2000-len(X1_te_temp[-1])))
    X1_te_temp[-1] = np.array(X1_te_temp[-1])
    i+=1
X1_te_temp = np.array(X1_te_temp)

X1_temp_oneHot = np.array(tf.one_hot(X1_temp,len(embedding_keys))[:,:,1:])
X1_val_temp_oneHot = np.array(tf.one_hot(X1_val_temp,len(embedding_keys))[:,:,1:])
X1_te_temp_oneHot = np.array(tf.one_hot(X1_te_temp,len(embedding_keys))[:,:,1:])

X4 = X4[:,1:701,:]
X4_val = X4_val[:,1:701,:]
X4_te = X4_te[:,1:701,:]

X2 = X2[:,1:701,:]
X2_val = X2_val[:,1:701,:]
X2_te = X2_te[:,1:701,:]

Y = Y[:,1:701,:]
Y_val = Y_val[:,1:701,:]
Y_te = Y_te[:,1:701,:]

100%|████████████████████████████████████| 12448/12448 [00:26<00:00, 474.15it/s]
2023-04-30 15:14:42.994024: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 15:14:43.012101: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 15:14:43.012834: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 15:14:43.013802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX 

In [6]:
class embeddingLoader(tfk.utils.Sequence):
    def __init__(self,path,pssms, masks,Y_oneHot,batch_size=30,shuffle=True):
        self.path = path
        self.pssms = pssms
        self.masks = masks
        self.Y_oneHot = copy.deepcopy(np.array(Y_oneHot))
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.length = len(Y_oneHot)
        self.indices = np.arange(0,self.length)
        self.on_epoch_end()

    def generate_batches(self):
        self.Y_oneHot_ = self.Y_oneHot[self.indices]
        self.pssms_ = self.pssms[self.indices]
        self.masks_ = self.masks[self.indices]
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        self.generate_batches()

    def __getitem__(self,index):
        token_representations = []
        for idx in self.indices[index*self.batch_size:(index+1)*self.batch_size]:
            token_representations.append(np.load(self.path+str(idx).zfill(6)+'.npy')[0])
        return ([
                tf.convert_to_tensor(np.array(token_representations)),
                self.masks_[index*self.batch_size:(index+1)*self.batch_size],
                self.pssms_[index*self.batch_size:(index+1)*self.batch_size]
                ],
                self.Y_oneHot_[index*self.batch_size:(index+1)*self.batch_size]
                )

    def __len__(self):

        if self.length%self.batch_size!=0:
            return 1+(self.length//self.batch_size)
        return self.length//self.batch_size

In [7]:
with strategy.scope():
    input1_ = tfk.layers.Input(shape=(700,300, ), name='sequence_input')
    input2_ = tfk.layers.Input(shape=(700,1, ), name='mask')
    input3_ = tfk.layers.Input(shape=(700,22, ), name='pssm')
    
    x = tfk.layers.Multiply()([input1_, input2_])
    x = tfk.layers.concatenate([x, input3_], axis=-1)
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru2')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output2')(x)
    output_ = tfk.layers.TimeDistributed(tfk.layers.Dense(8, activation='softmax') ,name='output')(x)
    
    conv_model = tfk.models.Model([input1_, input2_, input3_], output_)
    conv_model.compile(loss=NonZeroCCE(), metrics=[Accuracy], optimizer='adam', run_eagerly = True)
    conv_model.summary()
conv_model.load_weights("./Weights/conv_pssm_model.h5")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 700, 300)]   0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 700, 1)]     0           []                               
                                                                                                  
 multiply (Multiply)            (None, 700, 300)     0           ['sequence_input[0][0]',         
                                                                  'mask[0][0]']                   
                                                                                                  
 pssm (InputLayer)              [(None, 700, 22)]    0           []                           

In [8]:
print("*******************************************Conv ONLY*******************************************")
print()
print()
val_loader = embeddingLoader("./Data/Embeddings/Conv/val/",X2_val,X4_val,Y_val,30,False)
print("*******************************************VALIDATION*******************************************")
preds = conv_model.predict(val_loader, verbose=1)

m = np.sum(Y_val, axis=-1)
y_t = np.argmax(Y_val[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p,zero_division=0))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted',zero_division=0),
      recall_score(y_t,y_p,average='weighted',zero_division=0), f1_score(y_t,y_p,average='weighted',zero_division=0),f1_score(to_q3(y_t),to_q3(y_p),average='weighted',zero_division=0))

test_loader = embeddingLoader("./Data/Embeddings/Conv/test/",X2_te, X4_te,Y_te,30,False)
print()
print()
print("*******************************************Test*******************************************")
preds = conv_model.predict(test_loader, verbose=1)
m = np.sum(Y_te, axis=-1)
y_t = np.argmax(Y_te[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p,zero_division=0))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted',zero_division=0),
      recall_score(y_t,y_p,average='weighted',zero_division=0), f1_score(y_t,y_p,average='weighted',zero_division=0),f1_score(to_q3(y_t),to_q3(y_p),average='weighted',zero_division=0))


*******************************************Conv ONLY*******************************************


*******************************************VALIDATION*******************************************
 1/15 [=>............................] - ETA: 7s

2023-04-30 15:25:04.758635: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


15/15 [==============================] - 2s 135ms/step
              precision    recall  f1-score   support

           0       0.67      0.73      0.70      8216
           1       0.31      0.05      0.09       833
           2       0.82      0.87      0.84     19387
           3       0.40      0.25      0.31      3100
           4       0.42      0.10      0.16       551
           5       0.85      0.89      0.87     33679
           6       0.49      0.47      0.48      8082
           7       0.56      0.51      0.53      9997

    accuracy                           0.75     83845
   macro avg       0.56      0.48      0.50     83845
weighted avg       0.73      0.75      0.74     83845

0.828254517263999 0.7463176098753652 0.731377393381401 0.7463176098753652 0.7360303510987957 0.8281389416956056


*******************************************Test*******************************************
67/67 [==============================] - 9s 136ms/step
              precision    recall 

In [9]:
with strategy.scope():
    input1_ = tfk.layers.Input(shape=(700,300, ), name='sequence_input')
    input2_ = tfk.layers.Input(shape=(700,1, ), name='mask')
    input3_ = tfk.layers.Input(shape=(700,22, ), name='pssm')
    
    x = tfk.layers.Multiply()([input1_, input2_])
    x = tfk.layers.concatenate([x, input3_], axis=-1)
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru2')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output2')(x)
    output_ = tfk.layers.TimeDistributed(tfk.layers.Dense(8, activation='softmax') ,name='output')(x)

    gru_model = tfk.models.Model([input1_, input2_, input3_], output_)
    gru_model.compile(loss=NonZeroCCE(), metrics=[Accuracy], optimizer='adam', run_eagerly = True)
    gru_model.summary()
gru_model.load_weights("./Weights/gru_pssm_model.h5")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 700, 300)]   0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 700, 1)]     0           []                               
                                                                                                  
 multiply_4 (Multiply)          (None, 700, 300)     0           ['sequence_input[0][0]',         
                                                                  'mask[0][0]']                   
                                                                                                  
 pssm (InputLayer)              [(None, 700, 22)]    0           []                         

In [10]:
print("*******************************************GRU ONLY*******************************************")
print()
print()
val_loader = embeddingLoader("./Data/Embeddings/GRU/val/",X2_val,X4_val,Y_val,30,False)
print("*******************************************VALIDATION*******************************************")
preds = gru_model.predict(val_loader, verbose=1)

m = np.sum(Y_val, axis=-1)
y_t = np.argmax(Y_val[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p,zero_division=0))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted',zero_division=0),
      recall_score(y_t,y_p,average='weighted',zero_division=0), f1_score(y_t,y_p,average='weighted',zero_division=0),f1_score(to_q3(y_t),to_q3(y_p),average='weighted',zero_division=0))

test_loader = embeddingLoader("./Data/Embeddings/GRU/test/",X2_te, X4_te,Y_te,30,False)
print()
print()
print("*******************************************Test*******************************************")
preds = gru_model.predict(test_loader, verbose=1)
m = np.sum(Y_te, axis=-1)
y_t = np.argmax(Y_te[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p,zero_division=0))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted',zero_division=0),
      recall_score(y_t,y_p,average='weighted',zero_division=0), f1_score(y_t,y_p,average='weighted',zero_division=0),f1_score(to_q3(y_t),to_q3(y_p),average='weighted',zero_division=0))


*******************************************GRU ONLY*******************************************


*******************************************VALIDATION*******************************************
15/15 [==============================] - 2s 136ms/step
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      8216
           1       0.39      0.03      0.05       833
           2       0.78      0.90      0.84     19387
           3       0.41      0.24      0.30      3100
           4       0.50      0.09      0.15       551
           5       0.83      0.93      0.87     33679
           6       0.55      0.40      0.46      8082
           7       0.60      0.48      0.53      9997

    accuracy                           0.76     83845
   macro avg       0.60      0.47      0.49     83845
weighted avg       0.73      0.76      0.74     83845

0.8299123382431869 0.755835172043652 0.732023508999256 0.755835172043652 0.7363372829547511 0.826752

In [11]:
with strategy.scope():
    input1_ = tfk.layers.Input(shape=(700,600, ), name='sequence_input')
    input2_ = tfk.layers.Input(shape=(700,1, ), name='mask')
    input3_ = tfk.layers.Input(shape=(700,22, ), name='pssm')
    
    x = tfk.layers.Multiply()([input1_, input2_])
    x = tfk.layers.concatenate([x, input3_], axis=-1)
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru2')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output2')(x)
    output_ = tfk.layers.TimeDistributed(tfk.layers.Dense(8, activation='softmax') ,name='output')(x)

    conv_GRU_model = tfk.models.Model([input1_, input2_, input3_], output_)
    conv_GRU_model.compile(loss=NonZeroCCE(), metrics=[Accuracy], optimizer='adam', run_eagerly = True)
    conv_GRU_model.summary()
conv_GRU_model.load_weights("./Weights/conv_gru_pssm_model.h5")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 700, 600)]   0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 700, 1)]     0           []                               
                                                                                                  
 multiply_8 (Multiply)          (None, 700, 600)     0           ['sequence_input[0][0]',         
                                                                  'mask[0][0]']                   
                                                                                                  
 pssm (InputLayer)              [(None, 700, 22)]    0           []                         

In [12]:
print("*******************************************Conv_GRU ONLY*******************************************")
print()
print()
val_loader = embeddingLoader("./Data/Embeddings/Conv_GRU/val/",X2_val,X4_val,Y_val,30,False)
print("*******************************************VALIDATION*******************************************")
preds = conv_GRU_model.predict(val_loader, verbose=1)

m = np.sum(Y_val, axis=-1)
y_t = np.argmax(Y_val[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p,zero_division=0))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted',zero_division=0),
      recall_score(y_t,y_p,average='weighted',zero_division=0), f1_score(y_t,y_p,average='weighted',zero_division=0),f1_score(to_q3(y_t),to_q3(y_p),average='weighted',zero_division=0))

test_loader = embeddingLoader("./Data/Embeddings/Conv_GRU/test/",X2_te, X4_te,Y_te,30,False)
print()
print()
print("*******************************************Test*******************************************")
preds = conv_GRU_model.predict(test_loader, verbose=1)
m = np.sum(Y_te, axis=-1)
y_t = np.argmax(Y_te[m==1],axis=-1)
y_p = np.argmax(preds[m==1],axis=-1)
print(classification_report(y_t,y_p,zero_division=0))
print(accuracy_score(to_q3(y_t),to_q3(y_p)),accuracy_score(y_t,y_p),precision_score(y_t,y_p,average='weighted',zero_division=0),
      recall_score(y_t,y_p,average='weighted',zero_division=0), f1_score(y_t,y_p,average='weighted',zero_division=0),f1_score(to_q3(y_t),to_q3(y_p),average='weighted',zero_division=0))

*******************************************Conv_GRU ONLY*******************************************


*******************************************VALIDATION*******************************************
15/15 [==============================] - 3s 183ms/step
              precision    recall  f1-score   support

           0       0.75      0.68      0.71      8216
           1       0.41      0.04      0.08       833
           2       0.85      0.83      0.84     19387
           3       0.39      0.24      0.30      3100
           4       0.42      0.11      0.17       551
           5       0.80      0.94      0.86     33679
           6       0.56      0.37      0.45      8082
           7       0.54      0.54      0.54      9997

    accuracy                           0.75     83845
   macro avg       0.59      0.47      0.49     83845
weighted avg       0.73      0.75      0.73     83845

0.8280994692587512 0.7492038881269009 0.7303223747213653 0.7492038881269009 0.7325560431730684 